### <u> Comprehensive Data Modeling and Analysis with _Python_, _PostgreSQL_, _SQL_, and _Power BI_ </u>

In [17]:
#installing psycopg2 module for connecting to sql
!pip install psycopg2

#### Libraries

In [59]:
#Importing relevant libraries 
import pandas as pd
import psycopg2 

#### Making a function for _creating a DB and creating Tables_

In [986]:
#Creating a function that will create a DB 
def create_database():
    
    #connecting to default database
    con = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=Root")
    con.set_session(autocommit=True)
    cur = con.cursor()
    
    #creating a new database to store tables in.
    cur.execute("DROP DATABASE IF EXISTS ecommerce")
    cur.execute("CREATE DATABASE ecommerce")
    
    #close connection to default DB
    con.close()
    
    #connecting to ecommerce database
    con = psycopg2.connect("host=127.0.0.1 dbname=ecommerce user=postgres password=Root")
    cur=con.cursor()
    
    return cur, con 
    

In [987]:
#Creating a function that will drop a table 
def drop_table(cur,con):
    for query in drop_table_queries:
        cur.execute(query)
        con.commit()

In [988]:
#Creating a function that will create tables
def create_table(cur,con):
    for query in create_table_queries:
        cur.execute(query)
        con.commit()

#### Using pandas to pick specific columns from the csv's 
> You'll notice that some columns picked from the original csv are the same. 
>
> This is because ill be using a loop (iterow) to pick the rows inside the specified columns.

In [989]:
#Calling the fact table csv
fact = pd.read_csv("fact_table.csv")
fact.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0


In [990]:
#replacing all '_key' to 'id'
fact = fact.rename(columns={'coustomer_key': 'customer_id',
                            'payment_key':'payment_id',
                            'time_key':'time_id',
                            'item_key':'item_id',
                            'store_key': 'store_id'})

In [991]:
#Selecting columns needed
fact_cols = fact[['payment_id', 'customer_id', 'time_id', 'item_id', 'store_id', 'quantity', 'unit', 'unit_price', 'total_price' ]]
fact_cols.head(2)

,payment_id,customer_id,time_id,item_id,store_id,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0


In [992]:
#Calling the customer_dimension csv to pick the specific columns needed
customer_dim = pd.read_csv("customer_dim.csv", encoding='latin-1')
customer_dim.head(2)

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842


In [993]:
#Renaming customer column 'key' to 'id'
customer_dim = customer_dim.rename(columns={'coustomer_key': 'customer_id'})

In [994]:
#selecting columns needed
customer_dim_cols = customer_dim[["customer_id", "name", "contact_no"]]
customer_dim_cols.head(2)

,customer_id,name,contact_no
0,C000001,sumit,8801920345851
1,C000002,tammanne,8801817069329


In [995]:
#Calling the item_dimension csv to pick the specific columns needed
item_dim = pd.read_csv("item_dim.csv", encoding='latin=-1')
item_dim.head(2)

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans


In [996]:
#Renaming desc column to description & item_key to item_id  
item_dim = item_dim.rename(columns={'desc': 'description', 
                                    'item_key':'item_id'})

In [997]:
#selecting columns needed
item_dim_cols = item_dim[["item_id", "item_name", "description", "unit_price", "man_country", "supplier", "unit"]]
item_dim_cols.head(2)

,item_id,item_name,description,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans


In [998]:
#calling the store dimension table
store_dim = pd.read_csv("store_dim.csv")
store_dim.head(2)

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL


In [999]:
#Changing store_key to store_id
store_dim = store_dim.rename(columns={'store_key': 'store_id'})

In [1000]:
#Selecting specific columns
store_dim_cols = store_dim[["store_id", "division", "district"]]
store_dim_cols.head(2)

,store_id,division,district
0,S0001,SYLHET,HABIGANJ
1,S0002,SYLHET,HABIGANJ


In [1001]:
#Calling time dimension table
time_dim = pd.read_csv("time_dim.csv")
time_dim.head(2)

,time_key,date,hour,day,week,month,quarter,year
0,T00001,20-05-2017 14:56,14,20,3rd Week,5,Q2,2017
1,T00002,30-01-2015 22:14,22,30,4th Week,1,Q1,2015


In [1002]:
#Changing time_key to time_id
time_dim = time_dim.rename(columns={'time_key': 'time_id'})

In [1003]:
#Changing the format of the date column to allow sql to accept it without errors 
time_dim['date'] = pd.to_datetime(time_dim['date'], format='%d-%m-%Y %H:%M')

In [1004]:
#Confirming change was made
time_dim.head(1)

,time_id,date,hour,day,week,month,quarter,year
0,T00001,2017-05-20 14:56:00,14,20,3rd Week,5,Q2,2017


In [1005]:
#Selecting specific columns
time_dim_cols = time_dim[["time_id", "date", "hour", "day", "week", "month", "quarter", "year"]]
time_dim_cols.head(2)

,time_id,date,hour,day,week,month,quarter,year
0,T00001,2017-05-20 14:56:00,14,20,3rd Week,5,Q2,2017
1,T00002,2015-01-30 22:14:00,22,30,4th Week,1,Q1,2015


In [1006]:
#Calling transaction dimension table
trans_dim = pd.read_csv("Trans_dim.csv")
trans_dim.head(2)

,payment_key,trans_type,bank_name
0,P001,cash,NaN
1,P002,card,AB Bank Limited


In [1007]:
#Changing payment_key to payment_id
trans_dim = trans_dim.rename(columns={'payment_key': 'payment_id'})

In [1008]:
#Selecting specfic columns
trans_dim_cols = trans_dim[["payment_id", "trans_type", "bank_name"]]
trans_dim_cols.head(2)

,payment_id,trans_type,bank_name
0,P001,cash,NaN
1,P002,card,AB Bank Limited


In [1009]:
try:
    cur, con = create_database()
except psycopg2.OperationalError as e:
    # Exception occurred (session is still active)
    print(f"Error: {e}")
    
    # Add code to terminate the active session
    try:
        with con.cursor() as cursor:
            query =("SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE pid <> pg_backend_pid() AND datname = 'ecommerce';")
            cursor.execute(query)
            con.commit()
        print("Active session terminated successfully.")
    except Exception as ex:
        print(f"Error terminating session: {ex}")
else:
    # No exception occurred, database created successfully
    print("Database created successfully.")
finally:
    #Close the cursor and connection
    if cur:
        cur.close()
    if con:
        con.close()

Error: database "ecommerce" is being accessed by other users
DETAIL:  There is 1 other session using the database.

Error terminating session: current transaction is aborted, commands ignored until end of transaction block



In [1010]:
cur, con = create_database()

#### Creating tables using the columns selected
> **Note**: I cant start with the fact table since it has Foreign keys representing other tables.
ill start with the dimension tables first then finish with the fact table to avoid errors.

- _'Customer' table_

In [1011]:
#Creating customer table
customer_table = ("""CREATE TABLE IF NOT EXISTS customer(
    customer_id VARCHAR PRIMARY KEY,
    name VARCHAR, 
    contact_no BIGINT
)""")

#Executing the sql query 
cur.execute(customer_table)
con.commit()

- _'Item' table_

In [1012]:
#Creating item table
Item_table = ("""CREATE TABLE IF NOT EXISTS Item(
    item_id VARCHAR PRIMARY KEY,
    item_name VARCHAR,
    description VARCHAR,
    unit_price DECIMAL (10, 2),
    man_country VARCHAR,
    supplier VARCHAR,
    unit VARCHAR
)""")

#Executing the sql query 
cur.execute(Item_table)
con.commit()

- _'Store' table_

In [1013]:
#Creating Store table
store_table = ("""CREATE TABLE IF NOT EXISTS Store(
    store_id VARCHAR PRIMARY KEY,
    division VARCHAR, 
    district VARCHAR
)""")

#Executing the sql query 
cur.execute(store_table)
con.commit()

- _'Time' table_

In [1014]:
#Creating Time table
time_table = ("""CREATE TABLE IF NOT EXISTS Time(
    time_id VARCHAR PRIMARY KEY,
    date TIMESTAMP, 
    hour INT,
    day INT,
    week VARCHAR,
    month INT,
    quarter VARCHAR,
    year INT
)""")

#Executing the sql query 
cur.execute(time_table)
con.commit()

- _'Transaction' table_

In [1015]:
#Creating Transaction table
transaction_table = ("""CREATE TABLE IF NOT EXISTS Transaction(
    payment_id VARCHAR PRIMARY KEY,
    trans_type VARCHAR,
    bank_name VARCHAR
)""")

#Executing sql query
cur.execute(transaction_table)
con.commit()

- **_Fact_table_**
> **Note**: the fact table is called 'Sales' in the query for easier quering.

In [1016]:
fact_table = ("""CREATE TABLE IF NOT EXISTS Sales(
    payment_id VARCHAR,
    customer_id VARCHAR,
    time_id VARCHAR,
    item_id VARCHAR,
    store_id VARCHAR,
    quantity INT,
    unit VARCHAR,
    unit_price DECIMAL(10, 2),
    total_price DECIMAL(10, 2),
    FOREIGN KEY (payment_id) REFERENCES Transaction(payment_id),
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id),
    FOREIGN KEY (time_id) REFERENCES Time(time_id),
    FOREIGN KEY (item_id) REFERENCES Item(item_id),
    FOREIGN KEY (store_id) REFERENCES Store(store_id)
)""")

#Executing the sql query
cur.execute(fact_table)
con.commit() 

#### Inserting the rows respective to their column names.

- _'Customer' table_

In [1017]:
customer_table_insert = ("""INSERT INTO customer (
    customer_id,
    name,
    contact_no) 
VALUES (%s, %s, %s)
""")

for i, row in customer_dim_cols.iterrows():
    cur.execute(customer_table_insert, list(row))

con.commit()


- _'Item' table_

In [1018]:
item_table_insert = ("""INSERT INTO Item (
    item_id,
    item_name,
    description,
    unit_price,
    man_country,
    supplier,
    unit) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
""")

for i, row in item_dim_cols.iterrows():
    cur.execute(item_table_insert, list(row))

con.commit()

- _'Store' table_

In [1019]:
store_table_insert = ("""INSERT INTO Store (
    store_id,
    division,
    district )
VALUES (%s, %s, %s)
""")

for i, row in store_dim_cols.iterrows():
    cur.execute(store_table_insert, list(row))

con.commit()

- _'Time' table_

In [1020]:
time_dim_cols.head(1)

,time_id,date,hour,day,week,month,quarter,year
0,T00001,2017-05-20 14:56:00,14,20,3rd Week,5,Q2,2017


In [1021]:
time_table_insert = ("""INSERT INTO Time (
    time_id,
    date,
    hour,
    day,
    week,
    month,
    quarter,
    year) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""")

for i, row in time_dim_cols.iterrows():
    cur.execute(time_table_insert, list(row))

con.commit()

- _'Transacrion' table_

In [1022]:
transaction_table_insert = ("""INSERT INTO Transaction (
    payment_id,
    trans_type,
    bank_name) 
VALUES (%s, %s, %s)
""")

for i, row in trans_dim_cols.iterrows():
    cur.execute(transaction_table_insert, list(row))

con.commit()

- _'Fact' table_
> Sales 

In [1023]:
fact_table_insert = ("""INSERT INTO Sales (
    payment_id,
    customer_id,
    time_id,
    item_id,
    store_id,
    quantity,
    unit,
    unit_price,
    total_price) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""")

for i, row in fact_cols.iterrows():
    cur.execute(fact_table_insert, list(row))

con.commit()